# Sobol Sensitivity Analysis

In this notebook we apply the Sobol Sensitivy Analysis method to a building design problem.  
We determine the sensitivty of the objective (electricty use) to each of the design parameters.

In [1]:
import numpy as np
import pandas as pd
import time

from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct,
                                              ConstantKernel)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from SALib.sample import saltelli as ssampling
from SALib.analyze import sobol as sanalysis

from besos import eppy_funcs as ef
import sampling as sampling        
from problem import EPProblem
from evaluator import EvaluatorEP
from evaluator import EvaluatorGeneric
from parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist

from parameter_sets import parameter_set

## Build an EnergyPlus Evaluator

In [2]:
parameters = parameter_set(7) # use a pre-defined parameter set
problem = EPProblem(parameters, ['Electricity:Facility'])
building = ef.get_building() # use the example building
evaluator = EvaluatorEP(problem, building)
inputs = sampling.dist_sampler(sampling.lhs, problem, 50) # get 50 samples of the input space

/home/evan/anaconda3/lib/python3.6/site-packages/besos/problem.py:77: RuntimeWarning: Duplicate names found. (duplicate, repetitions): [('Watts per Zone Floor Area', 2)]
Attempting to fix automatically
  warnings.warn(RuntimeWarning(f'Duplicate names found. (duplicate, repetitions): '


## Fit the Surrogate model

Evaluate the samples to get training data.

In [3]:
outputs = evaluator.df_apply(inputs, processes=1)

Set up the surrogate and fit it.

In [4]:
hyperparameters = {'kernel':[None,1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
                             1.0 * RationalQuadratic(length_scale=1.0, alpha=0.5),
                             #ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0))**2),
                             1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),]}
folds = 3
gp = GaussianProcessRegressor(normalize_y=True)
clf = GridSearchCV(gp, hyperparameters, iid=True, cv=folds)

clf.fit(inputs, outputs)

print(f'The best performing model $R^2$ score on the validation set: {clf.best_score_}')
print(f'The model $R^2$ parameters: {clf.best_params_}')
#print(f'The best performing model $R^2$ score on a separate test set: {clf.best_estimator_.score(test_in, test_out)}')

The best performing model $R^2$ score on the validation set: 0.9336100063832058
The model $R^2$ parameters: {'kernel': 1**2 * Matern(length_scale=1, nu=1.5)}


Make an `Evaluator`.

In [5]:
def evaluation_func(ind):
    return ((clf.predict([ind])[0][0],),())

GP_SM = EvaluatorGeneric(evaluation_func, problem)

## Sobol Analysis

We can now derive the Sobol indices of the given design parameters.  
This is a global variance-based sensitivity analysis method.  
The resulting indices tell us how much of the variance is explained by each of the inputs.  
Sobol analysis may be very sample intensive, with 1000 samples per input.  
Simulation-based analysis would be very time intensive, so in this example we use a surrogate model instead. [[1]] [[2]]

[1]: https://www.sciencedirect.com/science/article/pii/S1364032112007101
[2]: http://statweb.stanford.edu/~owen/pubtalks/siamUQ.pdf

In [6]:
names =[parameters[i].name for i in range(len(parameters))]
bounds=[[parameters[i].value_descriptor.min, parameters[i].value_descriptor.max] for i in range(len(parameters))]

problem = {
    'num_vars': len(parameters),
    'names': names,
    'bounds': bounds
                    }

X = np.round(ssampling.sample(problem, N=10000, calc_second_order = True), decimals=3)
inputs = pd.DataFrame(data=X,columns=names)

print(f'This Sobol analysis will require {len(inputs)} design evaulations for the analysis.')

This Sobol analysis will require 160000 design evaulations for the analysis.


In [7]:
outputs = GP_SM.df_apply(inputs)
Y=outputs.values.ravel()

In [8]:
now = time.time()
Si = sanalysis.analyze(problem, Y.ravel(), conf_level=0.95,print_to_console=True, parallel=True, n_processors=4)
print(time.time()-now)
#pd.DataFrame(data=Si['mu_star'], index=Si['names']).sort_values(by=0)

Parameter S1 S1_conf ST ST_conf
Wall conductivity -0.000006 0.000107 0.000017 0.000001
Attic thickness -0.000010 0.000033 0.000001 0.000000
U-Factor 0.003477 0.004185 0.022687 0.001280
Solar Heat Gain Coefficient 0.003647 0.001710 0.004683 0.000202
Watts per Zone Floor Area_0 0.500818 0.019234 0.520764 0.014470
Watts per Zone Floor Area_1 0.459587 0.018769 0.484513 0.013108
Window to Wall Ratio -0.000032 0.000624 0.000465 0.000024

Parameter_1 Parameter_2 S2 S2_conf
Wall conductivity Attic thickness 0.000011 0.000153
Wall conductivity U-Factor 0.000017 0.000159
Wall conductivity Solar Heat Gain Coefficient 0.000009 0.000155
Wall conductivity Watts per Zone Floor Area_0 0.000026 0.000198
Wall conductivity Watts per Zone Floor Area_1 0.000004 0.000163
Wall conductivity Window to Wall Ratio 0.000010 0.000153
Attic thickness U-Factor 0.000012 0.000058
Attic thickness Solar Heat Gain Coefficient 0.000012 0.000057
Attic thickness Watts per Zone Floor Area_0 0.000013 0.000052
Attic thickness 